### Saving ethogram labels from jaaba into DEG for training

In [1]:
from animal_soup import *

2023-05-28 12:01:46.240385: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 12:01:46.664464: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/clewis7/repos/animal-soup/animal_soup/batch_utils.py:75: UserWarning: registration of accessor <class 'animal_soup.batch_utils.PathsDataFrameExtension'> under name 'paths' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  @pd.api.extensions.register_dataframe_accessor("paths")
/home/clewis7/repos/animal-soup/animal_soup/batch_utils.py:80: UserWarning: registration of accessor <class 'animal_soup.batch_utils.PathsSeriesExtension'> under name 'paths' for type <class 'pandas.core.series.Series'> is 

In [2]:
parent_path = set_parent_raw_data_path('/data/caitlin/training-vids/')

In [3]:
animal_id = 'M235Slc17a7_Gtacr2'

In [4]:
session_id = '20170425'

In [5]:
df_path = '/data/caitlin/data-frames/deg-df'

In [6]:
df = create_df(df_path, remove_existing=True)

In [7]:
df.behavior.add_item(animal_id=animal_id, session_id=session_id)

/home/clewis7/repos/animal-soup/animal_soup/_behavior.py:332: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['animal_id', 'session_id', 'mat_file', 'session_vids', 'notes'], dtype='object')]

  self._dataframe.to_hdf(self._dataframe.paths.get_df_path(), key='df')


In [8]:
df

,animal_id,session_id,mat_file,session_vids,notes
0,M235Slc17a7_Gtacr2,20170425,/data/caitlin/training-vids/M235Slc17a7_Gtacr2...,[/data/caitlin/training-vids/M235Slc17a7_Gtacr...,None


In [9]:
container = df.behavior.view()

/home/clewis7/venvs/mescore/lib/python3.11/site-packages/ipydatagrid/datagrid.py:445: UserWarning: Index name of 'index' is not round-trippable.
  schema = pd.io.json.build_table_schema(dataframe)


RFBOutputContext()

/home/clewis7/repos/mesmerize-core/mesmerize_core/arrays/_video.py:87: UserWarning: min not implemented for LazyTiff, returning min of 0th index
  warn("min not implemented for LazyTiff, returning min of 0th index")
/home/clewis7/repos/mesmerize-core/mesmerize_core/arrays/_video.py:92: UserWarning: max not implemented for LazyTiff, returning min of 0th index
  warn("max not implemented for LazyTiff, returning min of 0th index")


RFBOutputContext()

In [10]:
container.show()

In [15]:
container.ethogram_array.T.shape

(1998, 6)

In [12]:
import numpy as np
from pathlib import Path
import pandas as pd
import os

In [13]:
pd.DataFrame(columns=["background","lift", "handopen", "grab", "sup", "atmouth", "chew"])

,background,lift,handopen,grab,sup,atmouth,chew


In [27]:
from tqdm import tqdm

In [38]:
mat_path = df.iloc[-1]["mat_file"]
training_folder = Path('/data/caitlin/deg-jaaba/DATA/')
for option in tqdm(container.trial_selector.options):
    index = int(option.split('_v')[-1]) - 1
    etho_array, behaviors = container._get_ethogram(index, mat_path)
    etho_df = pd.DataFrame(columns=["background","lift", "handopen", "grab", "sup", "atmouth", "chew"])
    for row in etho_array.T:
        if sum(row) > 0:
            background = 0
        else:
            background = 1
        row = np.insert(row, 0, background)
        etho_df.loc[etho_df.index.size] = row
    path = training_folder.joinpath(option).joinpath(option).with_name(f'{option}_labels').with_suffix('.csv')
    etho_df.to_csv(path_or_buf=path)

100%|███████████████████████████████████████████████████████████████████████████████| 70/70 [00:41<00:00,  1.69it/s]
